In [1]:
# Import dependencies
import requests
import os
import pprint
from datetime import datetime,timedelta
import json
import pandas as pd
from dateutil.parser import parse

In [2]:
# Set params for query
pax = input(' # of adults will be traveling?')
children = input(' # Children traveling with you?')
p0 = input(' Starting place') 
startdate = input('Trip start date (mm/dd/yyyy)')
destinations = input('List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3)')
destinations.replace(" ", "")
destinations1 = list(destinations.split(','))




 # of adults will be traveling?1
 # Children traveling with you?0
 Starting placemiami
Trip start date (mm/dd/yyyy)12/01/2019
List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3)nyc,3,boston,2,la,4


In [9]:

p1=(str(destinations1[0]),int(destinations1[1]))
p2 = (str(destinations1[2]), int(destinations1[3]))
p3 = (str(destinations1[4]), int(destinations1[5]))
res=[]

def changeorder(p1,p2,p3,startdate,p0,children,pax):
    global res

    start_date = parse(startdate)
    days_p1=p1[1]
    days_p2=p2[1]
    days_p3=p3[1]
    start_p1 = start_date + timedelta(days=days_p1)
    start_p2 = start_p1 + timedelta(days=days_p2)
    start_p3 = start_p2 + timedelta(days=days_p3)
    url = "https://apidojo-hipmunk-v1.p.rapidapi.com/flights/create-session"

    querystring = {"from1":p1[0],"from2":p2[0],"from3":p3[0],"to1":p2[0],"to2":p3[0],"to3":p0,"date1":start_p1,"date2":start_p2,"date3":start_p3,"infants_lap":"0","children":children,"seniors":"0","country":"US","from0":p0,"to0":p1[0],"date0":startdate,"pax":pax,"cabin":"Coach"}
    headers = {
        'x-rapidapi-host': "apidojo-hipmunk-v1.p.rapidapi.com",
        'x-rapidapi-key': "f52da07954msh79f61d44ab028c2p138041jsne9cd9d426802"
        }

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    #does stuff with response
    idnums=[]
    agonies=[]
    prices=[]

    for key,value in response['itins'].items():
        idnum=key
        agony=value['agony']
        price=value['price']
        idnums.append(idnum)
        agonies.append(agony)
        prices.append(price)
    df=pd.DataFrame()
    df['price']=prices
    df['agonies']=agonies
    df['id']=idnums 


    # Print DF with values
    df = df.sort_values(['price'], ascending=[True])
    df = df[:5]

    res.append([(p1,p2,p3),df])



    p1,p2,p3 = p2,p3,p1
    return p1,p2,p3
    
    

In [10]:
res=[]
for _ in range(3):
    p1,p2,p3=changeorder(p1,p2,p3,startdate,p0,children,pax)
res


[[(('nyc', 3), ('boston', 2), ('la', 4)),
        price      agonies                                id
  1137  628.0  1508.283047  7814cb00e7b63fd2d45d9c19aa0db4ca
  1550  628.0  1522.892992  2d01718fa77666cfe0e1b79b159d8ab5
  1257  628.0  1514.755849  d9f8fb1bb7ba6c0a71406131fecfc969
  1572  628.0  1516.900068  8c588033a442fb8f4dcc8884a826a6e8
  1574  628.0  1519.358423  694e30f0a4a418616e6e8034efd4f0d7],
 [(('boston', 2), ('la', 4), ('nyc', 3)),
        price      agonies                                id
  3666  559.0  1732.600775  d64f34973a848d494a90fc257466ec94
  3264  559.0  1738.337566  aac25366ee197332d534416c75f9f412
  1784  559.0  1738.865125  49d75c638009ff3ca9f316e499050e46
  3134  559.0  1739.466767  4aa766621487d9153d7ad4315061f398
  39    559.0  1739.466767  10425e10b0b7281a22211149fb2b6e59],
 [(('la', 4), ('nyc', 3), ('boston', 2)),
        price      agonies                                id
  1802  901.0  2095.306317  937a12c5e884ccc7138a7ea59fe01967
  3236  901.0  2

In [22]:
df1=pd.DataFrame(res[0][1])
df2=pd.DataFrame(res[1][1])
df3=pd.DataFrame(res[2][1])

In [25]:
df3

,price,agonies,id
1802,901.0,2095.306317,937a12c5e884ccc7138a7ea59fe01967
3236,901.0,2085.473159,a205bd217a2b40ee888a0181e6517058
1070,901.0,2086.754511,161975d06003b517c7cb775b3bb40508
264,901.0,2084.258632,e3b0eaebc085578f54d39470480edd65
3224,901.0,2084.258632,289a4c86ec07e68348a25d7b7d668429


In [ ]:
days_p1